# Add more than one categorical predictor

Let's explore the ability to add more than one categorical predictor to our model. This will then build into thinking through interactions. Let's load some new data. These are bike data from Philadelphia program, called [Indego](https://www.rideindego.com/about/data/). The data loaded below are from Q3 of 2023. 

Here is information about the columns in the data.

+ **trip_id:** Locally unique integer that identifies the trip
+ **duration:** Length of trip in minutes
+ **start_time:** The date/time when the trip began, presented in ISO 8601 format in local time
+ **end_time:** The date/time when the trip ended, presented in ISO 8601 format in local time
+ **start_station:** The station ID where the trip originated (for station name and more information on each station see the Station Table)
+ **start_lat:** The latitude of the station where the trip originated
+ **start_lon:** The longitude of the station where the trip originated
+ **end_station:** The station ID where the trip terminated (for station name and more information on each station see the Station Table)
+ **end_lat:** The latitude of the station where the trip terminated
+ **end_lon:** The longitude of the station where the trip terminated
+ **bike_id:**  Locally unique integer that identifies the bike
+ **plan_duration:** The number of days that the plan the passholder is using entitles them to ride; 0 is used for a single ride plan (Walk-up)
+ **trip_route_category:** “Round Trip” for trips starting and ending at the same station or “One Way” for all other trips
+ **passholder_type:** The name of the passholder’s plan
+ **bike_type:** The kind of bike used on the trip, including standard pedal-powered bikes or electric assist bikes


In [1]:
library(tidyverse)
library(ggformula)

theme_set(theme_bw(base_size = 18))

temp <- tempfile()
download.file("https://bicycletransit.wpenginepowered.com/wp-content/uploads/2023/10/indego-trips-2023-q3.zip", temp)
bike <- readr::read_csv(unz(temp, "indego-trips-2023-q3-2.csv")) %>%
   filter(duration <= 120 & passholder_type != 'Walk-up')
unlink(temp)

head(bike)
dim(bike)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.3     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: ggstance


Attaching package: 'ggstance'


The following objects are masked from 'package:ggplot2':

    GeomErrorbarh, geom_errorbarh


Loading required package: scales


Attaching package: 'scales'


The following object is masked from 'package:purrr':

    discard


The following object is masked from 'package:readr':

    col_factor


Loading required package: ggridges


New to ggformula?  Try the tutorials: 
	learnr::run_tutorial("introducti

ERROR: Error in open.connection(structure(4L, class = c("unz", "connection"), conn_id = <pointer: 0x22f>), : cannot open the connection


Suppose we were interested in the following research questions. 

+ Does the trip route, type of pass, and bike type explain variation in the duration the bike was rented?

Let's explore this descriptively. 

In [ ]:
gf_density(~ duration, data = bike) %>%
  gf_labs(x = "Duration, in minutes")

In [ ]:
gf_violin(bike_type ~ duration, data = bike, fill = 'gray85', draw_quantiles = c(0.1, 0.5, 0.9)) %>%
  gf_labs(x = "Duration, in minutes", y = "") 

In [ ]:
gf_violin(passholder_type ~ duration, data = bike, fill = 'gray85', draw_quantiles = c(0.1, 0.5, 0.9)) %>%
  gf_labs(x = "Duration, in minutes", y = "") 

In [ ]:
gf_violin(trip_route_category ~ duration, data = bike, fill = 'gray85', draw_quantiles = c(0.1, 0.5, 0.9)) %>%
  gf_labs(x = "Duration, in minutes", y = "") 

## Regression model with multiple categorical predictors

Similar to multiple linear regression, including multiple categorical predictors is similar to that case. The simplest model is the additive model, also commonly referred to as the main effect model. Let's start with two categorical attributes that take on two different values. 

$$
duration = \beta_{0} + \beta_{1} bike\_type + \beta_{2} trip\_route\_category + \epsilon
$$

In [ ]:
bike_lm <- lm(duration ~ bike_type + trip_route_category, data = bike)

broom::glance(bike_lm)
broom::tidy(bike_lm) |>
  mutate_if(is.double, round, 3)

In [ ]:
two_way_predict <- broom::augment(bike_lm) %>%
   distinct(bike_type, trip_route_category, .fitted)

two_way_predict

In [ ]:
mean_duration <- df_stats(duration ~ bike_type + trip_route_category, data =bike, mean, length)
mean_duration

In [ ]:
gf_point(.fitted ~ bike_type, color = ~ trip_route_category, data = two_way_predict, size = 5) %>%
  gf_line(size = 1.5, group = ~ trip_route_category) %>%
  gf_labs(x = "", y = "Model Predicted Values", color = 'Trip Route') %>%
  gf_point(mean ~ bike_type, color = ~ trip_route_category, data = mean_duration, shape = 15, size = 5)

Back to the coefficients to really understand what these are.

In [ ]:
broom::tidy(bike_lm) |>
  mutate_if(is.double, round, 3)

+ *Intercept*: The average bike rental duration for the reference group (electric bikes and one-way trips). 
+ *bike_typestandard*: This is like a slope, so for a one unit change in bike type (ie, moving from an electric bike to a standard bike), the estimated mean change in bike duration decreased by 0.375 minutes, holding other attributes constant.
+ *trip_route_categoryRound Trip*: This is again like a slope, for a one unit change in trip route (ie, moving from a one-way trip to a round trip), the estimated mean change in bike duraction increased by 7.246 minutes, holder other attributes constant.

These are like weighted marginal means, where the weighting is occuring due to different sample sizes among the groups. 

In [ ]:
count(bike, bike_type, trip_route_category)

In [ ]:
df_stats(duration ~ bike_type + trip_route_category, data = bike, mean) %>%
  pivot_wider(names_from = 'trip_route_category', values_from = "mean")

### Interaction Model

The interaction model expands on the main effect only model, by allowing the effect of one category to differ based on elements of the other category. One way to think about this in the case with two categorical attributes, is that the mean change differs based on levels of the second attribute. This model, in the case where the attributes are each two groups, adds one additional term.

$$
duration = \beta_{0} + \beta_{1} bike\_type + \beta_{2} trip\_route\_category + \beta_{3} bike\_type:trip\_route\_category + \epsilon
$$

In [ ]:
bike_lm_int <- lm(duration ~ bike_type + trip_route_category + bike_type:trip_route_category, data = bike)

broom::glance(bike_lm_int)
broom::tidy(bike_lm_int)  |>
  mutate_if(is.double, round, 3)

In [ ]:
two_way_predict_int <- broom::augment(bike_lm_int) %>%
   distinct(bike_type, trip_route_category, .fitted)

two_way_predict_int

In [ ]:
gf_point(.fitted ~ bike_type, color = ~ trip_route_category, data = two_way_predict_int, size = 5) %>%
  gf_line(size = 1.5, group = ~ trip_route_category) %>%
  gf_labs(x = "", y = "Model Predicted Values", color = 'Trip Route') %>%
  gf_point(mean ~ bike_type, color = ~ trip_route_category, data = mean_duration, shape = 15, size = 5)

Back to the coefficients and what do these mean here. 

In [ ]:
broom::tidy(bike_lm_int)  |>
  mutate_if(is.double, round, 3)

+ *Intercept*: The average bike rental duration for the reference group (electric bikes and one-way trips). 
+ *bike_typestandard*: This is like a slope, so for a one unit change in bike type (ie, moving from an electric bike to a standard bike), the estimated mean change in bike duration decreased by .05 minutes, holding other attributes constant.
+ *trip_route_categoryRound Trip*: This is again like a slope, for a one unit change in trip route (ie, moving from a one-way trip to a round trip), the estimated mean change in bike duraction increased by 4.3 minutes, holder other attributes constant.
+ *bike_typestandard:trip_route_categoryRound Trip*: This is the interaction effect and is the **additional** mean level change for standard bikes *and* round trips, holding other attributes constant.

We can get the estimated means for the 4 groups as follows:

$$
\hat{\mu}_{elec-1way} = 15.2
$$
$$
\hat{\mu}_{elec-RT} = 15.2 + 4.3
$$
$$
\hat{\mu}_{stand-1way} = 15.2 - .05
$$
$$
\hat{\mu}_{stand-RT} = 15.2 - 0.05 + 4.3 + 4.7
$$

### Example

What about the example with two categorical attributes bike type and passholder type? Fit this model below and interpret the parameter estimates, both one without and with interaction effects. 

## Second order (three-way) interaction

The more attributes that interact with one another makes the model more complicated and difficult to interpret. Still, let's try a second order or three way interaction. 

In [ ]:
bike_lm_3way <- lm(duration ~ bike_type * trip_route_category * passholder_type, data = bike)

broom::glance(bike_lm_3way)
broom::tidy(bike_lm_3way) |>
  mutate_if(is.double, round, 3)

Interpreting these coefficients can be challenging, visualizing them can be a more effective way to undersand the impact these may have. The following steps will be used to visualize these model results:

1. Generate model-implied or predicted values for each combination of model values
2. Plot those model implied means

In [ ]:
three_way_predict_int <- broom::augment(bike_lm_3way) %>%
   distinct(bike_type, trip_route_category, passholder_type, .fitted)

three_way_predict_int

In [ ]:
gf_point(.fitted ~ bike_type, color = ~ trip_route_category, data = three_way_predict_int, size = 5) %>%
  gf_line(size = 1.5, group = ~ trip_route_category) %>%
  gf_facet_wrap(~ passholder_type) %>%
  gf_labs(y = "", x = "Model Predicted Values", color = 'Trip Route')